In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
breastcancerdf=pd.read_csv('/Users/adelamar_ifds/breastcanceranalysis/breast-cancer.csv')

In [3]:
len(breastcancerdf)

569

In [4]:
breastcancerdf.diagnosis.unique()

array(['M', 'B'], dtype=object)

In [7]:
dataset = pd.read_csv('/Users/adelamar_ifds/breastcanceranalysis/breast-cancer.csv')
x = dataset.iloc[:, 1:-1].values
y = dataset.iloc[:, -1].values

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)